In [2]:
import os

In [ ]:
import polars as pl
from pathlib import Path

In [ ]:
DATA_DIR = Path("data/lahman_1871-2024_csv")

pitch = pl.read_csv(DATA_DIR / "Pitching.csv")

# Filter for modern era
pitch_modern = pitch.filter(pl.col("yearID") >= 1946)

# Aggregate runs + outs per year & league
era_group = (
    pitch_modern
    .group_by(["yearID", "lgID"])
    .agg([
        pl.col("R").sum().alias("total_R"),
        pl.col("IPouts").sum().alias("total_IPouts"),
    ])
    .with_columns(
        (pl.col("total_R") / pl.col("total_IPouts") * 27).alias("ERA")
    )
)

era_group.head()

yearID,lgID,total_R,total_IPouts,ERA
i64,str,i64,i64,f64
1962,"""NL""",7278,43470,4.520497
1972,"""AL""",6441,49961,3.480855
1955,"""AL""",5491,33080,4.481771
1981,"""AL""",6112,40379,4.086877
1957,"""AL""",5210,33380,4.2142


In [5]:
bat = pl.read_csv(DATA_DIR / "Batting.csv")

bat_modern = bat.filter(pl.col("yearID") >= 1946)

bat_group = (
    bat_modern
    .group_by(["yearID", "lgID"])
    .agg([
        pl.col("AB").sum().alias("total_AB"),
        pl.col("H").sum().alias("total_H"),
        pl.col("2B").sum().alias("total_2B"),
        pl.col("3B").sum().alias("total_3B"),
        pl.col("HR").sum().alias("total_HR"),
    ])
    .with_columns([
        # total bases = H + 2B + 2*3B + 3*HR
        (pl.col("total_H") + pl.col("total_2B") + 2*pl.col("total_3B") + 3*pl.col("total_HR")).alias("total_TB"),
    ])
    .with_columns([
        (pl.col("total_TB") / pl.col("total_AB")).alias("SLG")
    ])
)

bat_group.head()


yearID,lgID,total_AB,total_H,total_2B,total_3B,total_HR,total_TB,SLG
i64,str,i64,i64,i64,i64,i64,i64,f64
1951,"""NL""",42704,11088,1746,367,1024,16640,0.389659
2009,"""NL""",87884,22741,4606,561,2482,35915,0.408664
1969,"""AL""",65536,16120,2385,378,1649,24208,0.369385
1968,"""NL""",54913,13351,1995,359,891,18737,0.341212
1981,"""NL""",43654,11141,1881,354,719,15887,0.36393


In [ ]:
# import polars as pl

def dh_state_expr():
    return (
        pl.when(pl.col("yearID") < 1973)
          .then(pl.lit("No DH (both)"))
          .when((pl.col("yearID").is_between(1973, 2019)) & (pl.col("lgID") == "AL"))
          .then(pl.lit("DH in AL; NL no DH"))
          .when((pl.col("yearID").is_between(1973, 2019)) & (pl.col("lgID") == "NL"))
          .then(pl.lit("NL no DH"))
          .when(pl.col("yearID") == 2020)
          .then(pl.lit("Universal DH (2020)"))
          .when((pl.col("yearID") == 2021) & (pl.col("lgID") == "AL"))
          .then(pl.lit("DH in AL; NL no DH"))
          .when((pl.col("yearID") == 2021) & (pl.col("lgID") == "NL"))
          .then(pl.lit("NL no DH"))
          .when(pl.col("yearID") >= 2022)
          .then(pl.lit("Universal DH (2022+)"))
          .otherwise(pl.lit("Unknown"))
          .alias("DH_state")
    )


In [15]:
era_group = era_group.with_columns(dh_state_expr())
bat_group = bat_group.with_columns(dh_state_expr())

era_group.write_csv(DATA_DIR / "league_era_by_year.csv")
bat_group.write_csv(DATA_DIR / "league_slugging_by_year.csv")


In [ ]:
# Check saved ERA file
pl.read_csv(DATA_DIR / "league_era_by_year.csv").head()


yearID,lgID,total_R,total_IPouts,ERA,DH_state
i64,str,i64,i64,f64,str
1962,"""NL""",7278,43470,4.520497,"""No DH (both)"""
1972,"""AL""",6441,49961,3.480855,"""No DH (both)"""
1955,"""AL""",5491,33080,4.481771,"""No DH (both)"""
1981,"""AL""",6112,40379,4.086877,"""DH in AL; NL no DH"""
1957,"""AL""",5210,33380,4.2142,"""No DH (both)"""


In [ ]:
# Check saved Slugging file
pl.read_csv(DATA_DIR / "league_slugging_by_year.csv").head()


yearID,lgID,total_AB,total_H,total_2B,total_3B,total_HR,total_TB,SLG,DH_state
i64,str,i64,i64,i64,i64,i64,i64,f64,str
1951,"""NL""",42704,11088,1746,367,1024,16640,0.389659,"""No DH (both)"""
2009,"""NL""",87884,22741,4606,561,2482,35915,0.408664,"""NL no DH"""
1969,"""AL""",65536,16120,2385,378,1649,24208,0.369385,"""No DH (both)"""
1968,"""NL""",54913,13351,1995,359,891,18737,0.341212,"""No DH (both)"""
1981,"""NL""",43654,11141,1881,354,719,15887,0.36393,"""NL no DH"""
